## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [14]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt

In [16]:
# 讀取 diabetes 辨識資料集 (種類是回歸)
diabetes = datasets.load_diabetes()

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(diabetes.data, diabetes.target, test_size=0.2, random_state=42)

In [25]:
# 預設參數 n_neighbors=5 建立模型
clf = GradientBoostingRegressor(random_state=7)

# 訓練模型
clf.fit(x_train,y_train)

# 使用模型預測
y_pred = clf.predict(x_test)

# 評分
print(metrics.mean_squared_error(y_test, y_pred))

2937.666978349464


In [30]:
# 設定要訓練的超參數組合
n_estimators = [50, 150, 250, 290]
max_depth = [1, 3, 4, 5, 8]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)
param_grid

{'n_estimators': [50, 150, 250, 290], 'max_depth': [1, 3, 4, 5, 8]}

In [31]:
## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1,cv = 10)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Best Accuracy: -3255.781484 using {'max_depth': 1, 'n_estimators': 150}


[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   10.7s finished
C:\Users\user\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [32]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

print(metrics.mean_squared_error(y_test, y_pred))

2752.5822659460046
